In [2]:
from PIL import Image
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import cv2
import layoutparser as lp
from pdf2image import convert_from_path
from paddleocr import PaddleOCR, draw_ocr
import tensorflow as tf
import numpy as np
import pandas as pd
import os

In [29]:

# os.chdir("D:/img_to_text")
# pdf_files = [file for (path,dir,file) in os.walk(os.getcwd())]
# pdf_files = [file for file in pdf_files[0] if file[-3:] == 'pdf']

# for f in pdf_files:    
#     images = convert_from_path(f)
#     for i, img in enumerate(images):
#         if not os.path.isdir("D:/img_to_text/converted_images"):
#             os.mkdir("converted_images")
#         filename = './converted_images/'+f[:-4]+str(i)+'.jpg'
#         img.save(filename)
# # import paddleocr.ppstructure.layout.predict_layout as lp

images = [file for (path,dirs,file) in os.walk('D:/img_to_text/converted_images')]
images = [file for file in images[0] if file.endswith('jpg') and 'crop' not in file]

os.chdir('D:/img_to_text/converted_images')
for image_file in images:
    image = cv2.imread(image_file)

    image = image[..., ::-1]

    # load model
    model = lp.models.PaddleDetectionLayoutModel(config_path="lp://PubLayNet/ppyolov2_r50vd_dcn_365e_publaynet/config",
                                    threshold=0.5,
                                    label_map={0: "Text", 1: "Title", 2: "List", 3:"Table", 4:"Figure"},
                                    enforce_cpu=False,
                                    enable_mkldnn=True)#math kernel library
    # detect
    layout = model.detect(image)

    x_1=0
    y_1=0
    x_2=0
    y_2=0

    try:
        
        tables = []
        for l in layout:
            #print(l)
            if (l.type == 'Table') and (int(l.block.y_2) - int(l.block.y_1) >200) :
                x_1 = int(l.block.x_1)
                print(l.block.x_1)
                y_1 = int(l.block.y_1)
                x_2 = int(l.block.x_2)
                y_2 = int(l.block.y_2)

            break

        mod_file = image_file[:-4] + 'crop' +'.jpg'
        
        image = cv2.imread(image_file)
        
        cv2.imwrite(mod_file, image[y_1:y_2,x_1:x_2])

        ocr = PaddleOCR(lang='en')
        image_path = mod_file
        image_cv = cv2.imread(image_path)
        image_height = image_cv.shape[0]
        image_width = image_cv.shape[1]
        output = ocr.ocr(image_path)[0]

        boxes = [line[0] for line in output]
        texts = [line[1][0] for line in output]
        probabilities = [line[1][1] for line in output]
        image_boxes = image_cv.copy()

        for box,text in zip(boxes,texts):
            cv2.rectangle(image_boxes, (int(box[0][0]),int(box[0][1])), (int(box[2][0]),int(box[2][1])),(0,0,255),1)
            cv2.putText(image_boxes, text,(int(box[0][0]),int(box[0][1])),cv2.FONT_HERSHEY_SIMPLEX,1,(222,0,0),1)
        
        im = image_cv.copy()
            
        horiz_boxes = []
        vert_boxes = []

        for box in boxes:
            x_h, x_v = 0,int(box[0][0])
            y_h, y_v = int(box[0][1]),0
            width_h,width_v = image_width, int(box[2][0]-box[0][0])
            height_h,height_v = int(box[2][1]-box[0][1]),image_height

            horiz_boxes.append([x_h,y_h,x_h+width_h,y_h+height_h])
            vert_boxes.append([x_v,y_v,x_v+width_v,y_v+height_v])

            cv2.rectangle(im,(x_h,y_h), (x_h+width_h,y_h+height_h),(0,0,255),1)
            cv2.rectangle(im,(x_v,y_v), (x_v+width_v,y_v+height_v),(0,255,0),1)
            
        horiz_out = tf.image.non_max_suppression(
            horiz_boxes,
            probabilities,
            max_output_size = 1000,
            iou_threshold=0.1,
            score_threshold=float('-inf'),
            name=None
        )
        horiz_lines = np.sort(np.array(horiz_out))
        print(horiz_lines)

        im_nms = image_cv.copy()

        for val in horiz_lines:
            cv2.rectangle(im_nms, (int(horiz_boxes[val][0]),int(horiz_boxes[val][1])), (int(horiz_boxes[val][2]),int(horiz_boxes[val][3])),(0,0,255),1)
            
        vert_out = tf.image.non_max_suppression(
            vert_boxes,
            probabilities,
            max_output_size = 1000,
            iou_threshold=0.05,
            score_threshold=float('-inf'),
            name=None
        )
        vert_lines = np.sort(np.array(vert_out))

        for val in vert_lines:
            cv2.rectangle(im_nms, (int(vert_boxes[val][0]),int(vert_boxes[val][1])), (int(vert_boxes[val][2]),int(vert_boxes[val][3])),(255,0,0),1)
            
        out_array = [["" for i in range(len(vert_lines))] for j in range(len(horiz_lines))]
        print(np.array(out_array).shape)
        print(out_array)

        unordered_boxes = []

        for i in vert_lines:
            print(vert_boxes[i])
            unordered_boxes.append(vert_boxes[i][0])
            
        ordered_boxes = np.argsort(unordered_boxes)
        print(ordered_boxes)

        def intersection(box_1, box_2):
            return [box_2[0], box_1[1],box_2[2], box_1[3]]

        def iou(box_1, box_2):

            x_1 = max(box_1[0], box_2[0])
            y_1 = max(box_1[1], box_2[1])
            x_2 = min(box_1[2], box_2[2])
            y_2 = min(box_1[3], box_2[3])

            inter = abs(max((x_2 - x_1, 0)) * max((y_2 - y_1), 0))
            if inter == 0:
                return 0

            box_1_area = abs((box_1[2] - box_1[0]) * (box_1[3] - box_1[1]))
            box_2_area = abs((box_2[2] - box_2[0]) * (box_2[3] - box_2[1]))

            return inter / float(box_1_area + box_2_area - inter)

        for i in range(len(horiz_lines)):
            for j in range(len(vert_lines)):
                resultant = intersection(horiz_boxes[horiz_lines[i]], vert_boxes[vert_lines[ordered_boxes[j]]] )

                for b in range(len(boxes)):
                    the_box = [boxes[b][0][0],boxes[b][0][1],boxes[b][2][0],boxes[b][2][1]]
                    if(iou(resultant,the_box)>0.1):
                        out_array[i][j] = texts[b]
                    
        out_array=np.array(out_array)
        pd.DataFrame(out_array).to_csv(image_file[:-3]+'.csv')
    
    except Exception as e:
        print(e)

OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgcodecs\src\loadsave.cpp:801: error: (-215:Assertion failed) !_img.empty() in function 'cv::imwrite'

OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgcodecs\src\loadsave.cpp:801: error: (-215:Assertion failed) !_img.empty() in function 'cv::imwrite'

OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgcodecs\src\loadsave.cpp:801: error: (-215:Assertion failed) !_img.empty() in function 'cv::imwrite'

OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgcodecs\src\loadsave.cpp:801: error: (-215:Assertion failed) !_img.empty() in function 'cv::imwrite'

OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgcodecs\src\loadsave.cpp:801: error: (-215:Assertion failed) !_img.empty() in function 'cv::imwrite'

OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgcodecs\src\loadsave.cpp:801: error: (-215:Assertion failed) !_img.empty() in function 'cv::imwrite'

Open

: 

In [28]:
image_file = "D:/img_to_text/converted_images/001 ECM_568248_v1_Lab samples for Nitrate testing for South Waikato District Council102crop.jpg"
image = cv2.imread(image_file)

image = image[..., ::-1]

# load model
model = lp.models.PaddleDetectionLayoutModel(config_path="lp://PubLayNet/ppyolov2_r50vd_dcn_365e_publaynet/config",
                            threshold=0.5,
                            label_map={0: "Text", 1: "Title", 2: "List", 3:"Table", 4:"Figure"},
                            enforce_cpu=False,
                            enable_mkldnn=True)#math kernel library
# detect
layout = model.detect(image)

x_1=0
y_1=0
x_2=0
y_2=0

try:

    tables = []
    for l in layout:
        #print(l)
        if (l.type == 'Table') and (int(l.block.y_2) - int(l.block.y_1) >200) :
            x_1 = int(l.block.x_1)
            print(l.block.x_1)
            y_1 = int(l.block.y_1)
            x_2 = int(l.block.x_2)
            y_2 = int(l.block.y_2)

        break

    mod_file = image_file[:-4] + 'crop2' +'.jpg'

    image = cv2.imread(image_file)

    cv2.imwrite(mod_file, image[y_1:y_2,x_1:x_2])

    ocr = PaddleOCR(lang='en')
    image_path = mod_file
    image_cv = cv2.imread(image_path)
    image_height = image_cv.shape[0]
    image_width = image_cv.shape[1]
    output = ocr.ocr(image_path)[0]

    boxes = [line[0] for line in output]
    texts = [line[1][0] for line in output]
    probabilities = [line[1][1] for line in output]
    image_boxes = image_cv.copy()

    for box,text in zip(boxes,texts):
        cv2.rectangle(image_boxes, (int(box[0][0]),int(box[0][1])), (int(box[2][0]),int(box[2][1])),(0,0,255),1)
        cv2.putText(image_boxes, text,(int(box[0][0]),int(box[0][1])),cv2.FONT_HERSHEY_SIMPLEX,1,(222,0,0),1)

    im = image_cv.copy()
        
    horiz_boxes = []
    vert_boxes = []

    for box in boxes:
        x_h, x_v = 0,int(box[0][0])
        y_h, y_v = int(box[0][1]),0
        width_h,width_v = image_width, int(box[2][0]-box[0][0])
        height_h,height_v = int(box[2][1]-box[0][1]),image_height

        horiz_boxes.append([x_h,y_h,x_h+width_h,y_h+height_h])
        vert_boxes.append([x_v,y_v,x_v+width_v,y_v+height_v])

        cv2.rectangle(im,(x_h,y_h), (x_h+width_h,y_h+height_h),(0,0,255),1)
        cv2.rectangle(im,(x_v,y_v), (x_v+width_v,y_v+height_v),(0,255,0),1)
        
    horiz_out = tf.image.non_max_suppression(
        horiz_boxes,
        probabilities,
        max_output_size = 1000,
        iou_threshold=0.1,
        score_threshold=float('-inf'),
        name=None
    )
    horiz_lines = np.sort(np.array(horiz_out))
    print(horiz_lines)

    im_nms = image_cv.copy()

    for val in horiz_lines:
        cv2.rectangle(im_nms, (int(horiz_boxes[val][0]),int(horiz_boxes[val][1])), (int(horiz_boxes[val][2]),int(horiz_boxes[val][3])),(0,0,255),1)
        
    vert_out = tf.image.non_max_suppression(
        vert_boxes,
        probabilities,
        max_output_size = 1000,
        iou_threshold=0.1,
        score_threshold=float('-inf'),
        name=None
    )
    vert_lines = np.sort(np.array(vert_out))

    for val in vert_lines:
        cv2.rectangle(im_nms, (int(vert_boxes[val][0]),int(vert_boxes[val][1])), (int(vert_boxes[val][2]),int(vert_boxes[val][3])),(255,0,0),1)
        
    out_array = [["" for i in range(len(vert_lines))] for j in range(len(horiz_lines))]
    print(np.array(out_array).shape)
    print(out_array)

    unordered_boxes = []

    for i in vert_lines:
        print(vert_boxes[i])
        unordered_boxes.append(vert_boxes[i][0])
        
    ordered_boxes = np.argsort(unordered_boxes)
    print(ordered_boxes)

    def intersection(box_1, box_2):
        return [box_2[0], box_1[1],box_2[2], box_1[3]]

    def iou(box_1, box_2):

        x_1 = max(box_1[0], box_2[0])
        y_1 = max(box_1[1], box_2[1])
        x_2 = min(box_1[2], box_2[2])
        y_2 = min(box_1[3], box_2[3])

        inter = abs(max((x_2 - x_1, 0)) * max((y_2 - y_1), 0))
        if inter == 0:
            return 0

        box_1_area = abs((box_1[2] - box_1[0]) * (box_1[3] - box_1[1]))
        box_2_area = abs((box_2[2] - box_2[0]) * (box_2[3] - box_2[1]))

        return inter / float(box_1_area + box_2_area - inter)

    for i in range(len(horiz_lines)):
        for j in range(len(vert_lines)):
            resultant = intersection(horiz_boxes[horiz_lines[i]], vert_boxes[vert_lines[ordered_boxes[j]]] )

            for b in range(len(boxes)):
                the_box = [boxes[b][0][0],boxes[b][0][1],boxes[b][2][0],boxes[b][2][1]]
                if(iou(resultant,the_box)>0.1):
                    out_array[i][j] = texts[b]
                
    out_array=np.array(out_array)
    pd.DataFrame(out_array).to_csv(image_file[:-3]+'.csv', index = False,header= False)

except Exception as e:
    print(e)

8.644897
[2024/01/29 17:08:49] ppocr DEBUG: Namespace(alpha=1.0, benchmark=False, beta=1.0, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='C:\\Users\\Richard/.paddleocr/whl\\cls\\ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_box_type='quad', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='C:\\Users\\Richard/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='max', e2e_model_dir=None, e2e_pgnet_mode='fast', e2e_pgn

In [20]:
out_array

array([['', '', '', '', '', '', 'AR-17-NU-03735'],
       ['', '', '', '', '', '', ''],
       ['', '', '', '', '', '', ''],
       ['', '', '', '', '', '', ''],
       ['', '', '', '', '', '', '49263'],
       ['', '', '', '', '', '', '06/10/2017'],
       ['', '', '', '', '', '', '13/09/2017 06:45:'],
       ['', '', '', '', '', '', 'LOQCLIENT SPECIFICATIONS'],
       ['', '', '', '', '', '', ''],
       ['', '', '', '', '', '', ''],
       ['', '', '', '', '', '', ''],
       ['', '', '', '', '', '', ''],
       ['', '', '', '', '', '', ''],
       ['', '', '', '', '', '', ''],
       ['', '', '', '', '', '', ''],
       ['', '', '', '', '', '', ''],
       ['', '', '', '', '', '', ''],
       ['', '', '', '', '', '', ''],
       ['', '', '', '', '', '', ''],
       ['', '', '', '', '', '', ''],
       ['', '', '', '', '', '', ''],
       ['', '', '', '', '', '', ''],
       ['', '', '', '', '', '', ''],
       ['', '', '', '', '', '', ''],
       ['', '', '', '', '', '', ''],
     